<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">Trabajo Fin de Máster</p>
<p style="margin: 0; text-align:right;">Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Autor: César Fernández Domínguez</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Análisis descriptivo de datos de contaminación del aire para la ciudad de Barcelona

Este análisis se ha organizado en los siguientes apartados:

 <ol start="1">
  <li>Carga de datos</li>
  <li>Definición de modelos
  <br>2.1. Modelo de regresión de bosques aleatorios (Random Forest - RF) 
  <br>2.2. Modelo de regresión de máquina de vectores de soporte (Support Vector Machine - SVM) 
  <br>2.3. Modelo de regresión mediante red neuronal artificial de tipo perceptrón de multicapa (MultiLayer Perceptron - MLP) 
  <br>2.4. Modelo de regresión mediante una red neuronal recurrente Long Short-Term Memory (LSTM) 
  <br>2.5. Modelo de regresión mediante una red neuronal recurrente a base de Gated Recurrent Units (GRU) 
  <li>Selección de variables más representativas
  <br>3.1. Selección de variables mediante análisis de componentes principales (PCA)
  <br>3.2. Selección de variables mediante la técnica de Forward Selection (FS)
  <li>Ajuste de hiperparámetros
</ol>
   
En primer lugar, cargamos algunas librerías que vamos a necesitar, y definimos la carpeta raiz de donde cargaremos nuestros datos y, finalmente,  guardaremos los resultados:

In [1]:
# Mount folder in Google Drive (only for execution in Google Colab)
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import sys, os
import numpy as np
import pandas as pd

rootDataFolder="../data/"
#rootDataFolder="/content/drive/My Drive/colab/data/"

En este notebook, trataremos de analizar hasta que punto necesitariamos de todas las variables independientes, de que disponemos, para obtener un resultado óptimo con los modelos de predicción de contaminación del aire propuestos en el capítulo 2.

# 1. Carga de datos

Inicialmente, vamos a cargar los datos de contaminantes registrados, datos meteorológicos y datos de tránsito de vehículos, que previamente hemos analizado y preparado.

In [ ]:
# Read air data file
data_air_hourly = pd.read_csv(os.path.join(rootDataFolder, "data_air_hourly_red.csv"), sep=',', encoding = "UTF-8")
data_air_hourly=data_air_hourly.drop('PM10', axis=1)
data_air_hourly['DATA'] = pd.DatetimeIndex(data_air_hourly['DATA'])

# Read imputated air data file
data_air_hourly_imp = pd.read_csv(os.path.join(rootDataFolder, "data_air_hourly_imp.csv"), sep=',', encoding = "UTF-8")
data_air_hourly_imp=data_air_hourly_imp.drop('PM10', axis=1)
data_air_hourly_imp['DATA'] = pd.DatetimeIndex(data_air_hourly_imp['DATA'])

# Read air stations data file
data_air_stations = pd.read_csv(os.path.join(rootDataFolder, "data_air_stations.csv"), sep=',', encoding = "UTF-8")

# Read meteo data file
data_meteo_hourly = pd.read_csv(os.path.join(rootDataFolder, "data_meteo_hourly_red.csv"), sep=',', encoding = "UTF-8")
data_meteo_hourly = data_meteo_hourly.rename(columns={"data_lectura": "DATA"})
data_meteo_hourly = data_meteo_hourly.drop(['DVVx10','HRn','HRx','PPTx1min','Pn','Px','Tn','Tx','VVx10'], axis=1)
data_meteo_hourly['DATA'] = pd.DatetimeIndex(data_meteo_hourly['DATA'])

# Read trafic data file
data_trafic = pd.read_csv(os.path.join(rootDataFolder, "data_trafico_red.csv"), sep=',', encoding = "UTF-8")
data_trafic['DATA'] = pd.DatetimeIndex(data_trafic['DATA'])
data_trafic=data_trafic.sort_values(by=['DATA'])

Vemos que, prescindimos de la variable $PM_{10}$, puesto que solamente disponemos de datos desde el año 2012 en una de las estaciones (Eixample), en las demás no tendríamos datos hasta finales de 2015. Por esta razón, de momento, prescindiremos de este contaminante para nuestro estudio.

En el caso de los datos meteorológicos, prescindimos de aquellas variables que, como vimos en el análisis previo, pueden ser descartadas por existir otras con gran correlación con ellas. En todo caso, no sabemos a ciencia cierta cuales de estas variables, las descartadas o las mantenidas, tendrán mayor influencia en la precisión de nuestros modelos de predicción.

Dado que, para los datos meteorológicos, disponemos de tres estaciones meteorológicas y, que son cuatro las estaciones de observación de calidad del aire seleccionadas de la ciudad de Barcelona y, ya teniendo los datos de tránsito de vehículos agrupados por estas estaciones, definimos una función que realiza la unión de todos estos conjuntos de datos para una estación de observación de calidad del aire dada. A continuación, vemos el código fuente de esta función y la correspondencia dada entre estaciones meteorológicas y de calidad del aire.

In [ ]:
from functools import reduce

def get_station_data(station, data_air, data_meteo, data_trafic):
    '''
    Merge air, meteo and trafic data for a given station.
    
    Inputs:
    
        - station: air station code
        - data_air: dataframe with air data
        - data_meteo: dataframe with meteo data
        - data_trafic: dataframe with trafic data
        
    return dataframe with merged data for input station.
    '''

    # Correspondence between air and meteo stations
    air_to_meteo_station_lut = {
        8019054:'D5',
        8019043:'X4',8019044:'X4',
        8019057:'X8'}
    
    # Get meteo station for input air station code 
    codiStation=air_to_meteo_station_lut[station]
    
    # Select air, meteo and trafic data for given air station
    data_air_station=data_air.loc[data_air['CODI_EOI'] == station,].drop(['CODI_EOI'], axis=1)  
    data_meteo_station=data_meteo.loc[data_meteo['codi_estacio'] == codiStation,].drop(['codi_estacio'], axis=1)
    data_trafic_station=data_trafic.loc[data_trafic['CODI_EOI'] == station,].drop(['CODI_EOI'], axis=1)
    
    # Merge everything in a dataframe
    dfs=[data_air_station,data_meteo_station,data_trafic_station]
    data_station = reduce(lambda left,right: pd.merge(left, right, on=['DATA'], how='outer'), dfs)
    data_station = data_station.sort_values(by=['DATA'])
    
    return data_station

Seguidamente, a través del siguiente código, mostramos una tabla con la disponibilidad de datos para cada una de las variables cargadas.

In [14]:
data_air_in = data_air_hourly_imp
data_meteo_in = data_meteo_hourly
data_trafic_in = data_trafic
datas = pd.DataFrame()
stations=data_air_in['CODI_EOI'].unique()
for station in stations:

    row = {'CODI_EOI':[station]}
    row['NOM_ESTACIO']=[data_air_stations.loc[data_air_stations["CODI_EOI"] == station,'NOM_ESTACIO'].iloc[0]]
    row['TIPUS_ESTACIO']=[data_air_stations.loc[data_air_stations["CODI_EOI"] == station,'TIPUS_ESTACIO'].iloc[0]]
    row['AREA URBANA']=[data_air_stations.loc[data_air_stations["CODI_EOI"] == station,'AREA URBANA'].iloc[0]]

    data_in = get_station_data(station, data_air_in, data_meteo_in, data_trafic_in).set_index('DATA').resample('D').mean()
    variables = data_in.columns
    for variable in variables:
        ts = pd.Series(data_in[variable].values, index=data_in.index)
        #row[variable+"count"] = ts.count()
        first_valid_index = ts.first_valid_index()
        row[variable] = [None if first_valid_index is None else first_valid_index.strftime('%Y-%m-%d')]
        last_valid_index = ts.last_valid_index()
        #row[variable+'_last'] = [None if last_valid_index is None else last_valid_index.strftime('%Y-%m-%d')]
    datas = datas.append(pd.DataFrame.from_dict(row), ignore_index=True, sort=False)
datas.set_index('CODI_EOI')
datas

,CODI_EOI,NOM_ESTACIO,TIPUS_ESTACIO,AREA URBANA,CO,COcount,NO,NOcount,NO2,NO2count,NOX,NOXcount,O3,O3count,SO2,SO2count,DV10,DV10count,HR,HRcount,P,Pcount,PPT,PPTcount,RS,RScount,T,Tcount,VV10,VV10count,traffic,trafficcount
0,8019057,Barcelona (Palau Reial),background,urban,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2017-10-01,943
1,8019054,Barcelona (Parc Vall Hebron),background,urban,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2010-01-01,3764,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3765,2010-01-01,3771,2010-01-01,3767,2017-10-01,943
2,8019043,Barcelona (Eixample),traffic,urban,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2017-10-01,943
3,8019044,Barcelona (Gracia - Sant Gervasi),traffic,urban,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2012-01-01,3043,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2010-01-01,3771,2017-10-01,943


# 2. Definición de modelos

En este trabajo trataremos de encontrar un modelo de minería de datos que nos permita predecir algunos de los contaminantes que definen la calidad del aire en la ciudad de Barcelona. Para ello vamos a evaluar y comparar cinco modelos distintos, basados en técnicas muy diversas. Los modelos que hemos utilizado son: modelo de RandomForest (**RF**), máquina de vectores de soporte (en sus siglas en inglés: **SVM**), modelo de redes neuronales artificiales (**ANN**, del inglés) basado en perceptrón multicapa (**MLP**, del inglés) y modelos de redes neuronales recurrentes, como son **LSTM** y **GRU**.

Seguidamente, en los siguientes subapartados, incluimos una breve explicación de cada modelo propuesto y su implementación en *Python*.

## 2.1. Modelo de regresión de bosques aleatorios (Random Forest - RF) 

La técnica de Random Forest es un tipo de combinación de clasificadores que utiliza la estrategia denominada *bagging*, cuya idea principal reside en utilizar el conjunto de datos de entrenamiento para generar centenares o miles de conjunto similares con cada uno de los cuales se construye un regresor, cuyos resultados, de cada uno de ellos, se vuelve a incluir como entrada de un regresor final. Está técnica ofrece muy buenos resultados en problemas de clasificación. 

A continuación, incluimos una función que define el modelo de regresión para nuestro predictor basado en RandomForest. Como parámetro susceptible de ser parametrizado definimos la máxima profundidad de los árboles de decisión generados internamente en el modelo.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def RandomForest_Model(nro_x_columns, nro_y_columns, n_steps_in=1, n_steps_out=1, params={'max_depth':2}):
    '''
    Build prediction model based in RandomForest.
    
    Inputs:
    
        - nro_x_columns: number of columns used as independant variables (predictors).
        - nro_y_columns: number of columns used as dependant variables (predicted).
        - n_steps_in: number of backward steps considered to predictors.
        - n_steps_out: number of forward steps considered to predicted variables.
        - params: parameters to the model
        
    return 'model'.
    '''
    model = RandomForestRegressor(
               max_features=nro_x_columns*n_steps_in,
               n_estimators=nro_y_columns*n_steps_out,
               max_depth=int(params['max_depth']))
    return model

## 2.2. Modelo de regresión de máquina de vectores de soporte (Support Vector Machine - SVM) 

La técnica denominada de máquina de vectores de soporte se basa en la definición de un hiperplano óptimo en forma de superficie de decisión, de forma que el margen de separación entre los datos en cada uno de los lados del hiperplano se maximiza. Esta técnica fué desarrolla por *Vladimir Vapnik* y su equipo en los laboratorios *AT&T*. La gran aportación de Vapnik radica en que construye un método que tiene por objetivo producir predicciones en las que se puede tener mucha confianza, en lugar de lo que se ha hecho tradicionalmente, que consiste en construir hipótesis que cometan pocos errores.

La siguiente función define un modelo de regresión para nuestro predictor basado en **SVM**. Se permite parametrizar los parámetros del modelo **SVM**: el parámetro de regulación $C$ y el coeficiente del kernel $gamma$. En nuestro caso utilizaremos el kernel por defecto: *rbf* o radial.

In [ ]:

from sklearn.svm import SVR

def SVM_Model(nro_x_columns, nro_y_columns, n_steps_in=1, n_steps_out=1, params={'C':1.0, 'gamma':1.e-7}):
    '''
    Build prediction model based in SVM.
    
    Inputs:
    
        - nro_x_columns: number of columns used as independant variables (predictors).
        - nro_y_columns: number of columns used as dependant variables (predicted).
        - n_steps_in: number of backward steps considered to predictors.
        - n_steps_out: number of forward steps considered to predicted variables.
        - params: parameters to the model
        
    return 'model'.
    '''
    model = SVR(
        C=float(params['C']), # float
        gamma=float(params['gamma'])) # {‘scale’, ‘auto’} or float
    return model



## 2.3. Modelo de regresión mediante red neuronal artificial de tipo perceptrón de multicapa (MultiLayer Perceptron - MLP) 

Un perceptron multicapa (*MultiLayer Perceptron* - **MLP**) es una clase de red neuronal artificial (*ANN*, del inglés). Se denomina *Vanilla* cuando está construida con una única capa oculta. Al menos consiste de tres capas: una capa de entrada, una capa oculta, y una capa de salida o de activación. 

Utilizan el método de retropropagación (*backpropagation*) que se basa en inicialmente una propagación hacia adelante (*feedforward*) de las instancias de entrenamiento, obteniendo la salida de la red neuronal para, posteriormente, realizar una propagación hacia atrás (*backpropagation*) del error cometido en la capa de salida, el cual es propagado hacia atrás para calcular los valores delta de las neuronas de las capas ocultas. 

Normalmente el poder de aprendizaje de una red neuronal artificial viene determinado por el número de neuronas en la capa oculta. Sin embargo, este tipo de redes neuronales presentan el problema denominado de desaparición del gradiente (*the vanishing gradient problem*), por el cual produce que la red se vuelva inestable para valores relativamente altos de neuronas ocultas.

En la siguiente función se define una implementación de un modelo *Vanilla MLP*. En esta función podremos parametrizar el número de neuronas de la capa oculta. Durante el proceso de entrenamiento podremos también parametrizar el número de *epochs* y el *batch_size*.

In [10]:
from keras.models import Sequential
from keras.layers import Dense

def MLP_Model(nro_x_columns, nro_y_columns, n_steps_in=1, n_steps_out=1, params={'neurons':50}):
    '''
    Build prediction model based in MLP.
    
    Inputs:
    
        - nro_x_columns: number of columns used as independant variables (predictors).
        - nro_y_columns: number of columns used as dependant variables (predicted).
        - n_steps_in: number of backward steps considered to predictors.
        - n_steps_out: number of forward steps considered to predicted variables.
        - params: parameters to the model
        
    return 'model'.
    '''
    model = Sequential()
    model.add(Dense(int(params['neurons']), activation='relu', input_dim=nro_x_columns*n_steps_in))
    model.add(Dense(nro_y_columns*n_steps_out))
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return model     

Using TensorFlow backend.


## 2.4. Modelo de regresión mediante una red neuronal recurrente Long Short-Term Memory (LSTM) 

Las redes neuronales recurrentes (*Recurrent Neural Networks* - **RNN**), son un tipo de redes neuronales en las cuales se introducen bucles de retroalimentación. Estas redes surgen con la idea de modelar datos en forma de series temporales. La idea en estos modelos es tener neuronas que se activan durante un tiempo limitado. Esta activación puede estimular otras neuronas, que se pueden activar un poco más tarde, también por una duración limitada.

Una unidad **LSTM** comun esta formada por una celda, una puerta de entrada (*input gate*), una puerta de salida (*output gate*) y una puerta de olvido (*forget gate*). La celda recuerda valores sobre intervalos de tiempo arbitrarios y las tres puertas regulan el flujo de información de entrada y salida de la celda. Este tipo de estructuras permiten superar el problema de desaparición del gradiente presentado en las redes neuronales artificiales.

La siguiente función define un modelo basado en unidades neuronales **LSTM**. Se permiten parametrizar los mismos parámetros que para el modelo **MLP**.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

def LSTM_Model(nro_x_columns, nro_y_columns, n_steps_in=1, n_steps_out=1, params={'neurons':50}):
    '''
    Build prediction model based in LSTM.
    
    Inputs:
    
        - nro_x_columns: number of columns used as independant variables (predictors).
        - nro_y_columns: number of columns used as dependant variables (predicted).
        - n_steps_in: number of backward steps considered to predictors.
        - n_steps_out: number of forward steps considered to predicted variables.
        - params: parameters to the model
        
    return 'model'.
    '''
    model = Sequential()
    model.add(LSTM(int(params['neurons']), activation='relu', input_shape=(n_steps_in, nro_x_columns)))
    model.add(Dense(nro_y_columns*n_steps_out))
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return model

## 2.5. Modelo de regresión mediante una red neuronal recurrente a base de Gated Recurrent Units (GRU) 

Las unidades **GRU** son una extensión de la red **LSTM**. Estas añaden una puerta de actualización y de otra de olvido. En conjunto, estas puertas, contribuyen al equilibrio del flujo de datos dentro de la unidad.

Definimos la siguiente función para encapsular un modelo basado en celulas **GRU**. Los parámetros a parametrizar serán los mismos que para los otros dos modelos basados en redes neuronales.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU

def GRU_Model(nro_x_columns, nro_y_columns, n_steps_in=1, n_steps_out=1, params={'neurons':50}): 
    '''
    Build prediction model based in GRU.
    
    Inputs:
    
        - nro_x_columns: number of columns used as independant variables (predictors).
        - nro_y_columns: number of columns used as dependant variables (predicted).
        - n_steps_in: number of backward steps considered to predictors.
        - n_steps_out: number of forward steps considered to predicted variables.
        - params: parameters to the model
        
    return 'model'.
    '''
    model = Sequential()
    model.add(GRU(units=int(params['neurons']), input_shape=(n_steps_in, nro_x_columns)))
    model.add(Dense(nro_y_columns*n_steps_out, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return model    

# 3. Selección de variables más representativas

En el siguiente apartado, evaluaremos varios métodos para seleccionar, de nuestro conjunto de variables independientes, aquellas que resultan más representativas para la predicción de una determinada variable dependiente. A pesar de que los modelos basados en redes neuronales soportan bien el trabajar con un gran número de variables independientes, nos interesa limitar el número de estas para evitar, de alguna forma, un sobre-entrenamiento del modelo y un elevado tiempo de aprendizaje.

En nuestro caso, hemos utilizado dos métodos distintos para la selección de las variables independientes más representativas para cada caso. Un primer método se basa en el análisis de componentes principales (**PCA**) (para este método se han planteado dos opciones distintas para seleccionar las variables más representativas de nuestro conjunto de datos, como veremos más adelante). El segundo método de selección de variables utilizado, se trata de un método de tipo StepWise Regression, en concreto el método denominado Forward Selection. Los siguientes dos apartados tratan de explicar un poco más estos dos métodos.

## 3.1. Selección de variables mediante análisis de componentes principales (PCA)

Informalmente, definiremos el análisis de componentes principales como una técnica que intenta conseguir una representación de un conjunto de datos en un espacio de dimensionalidad más reducida, minimizando el error cuadrático cometido. 

El algoritmo **PCA** reduce la dimensionalidad mediante una transformación lineal que escoge un nuevo sistema de coordenadas para el conjunto original de datos (es decir, realiza una rotación del espacio d-dimensional), en el cual la varianza de mayor tamaño del conjunto de datos se recoge en el primer eje (llamado el primer componente principal), la segunda varianza más grande en el segundo eje, y así sucesivamente. Este algoritmo se basa en la matriz de covarianzas o correlaciones de los datos originales, de forma que calcula los vectores propios (*eigen* valares) de esta matriz y se aplica a los datos originales para conseguir la transformación lineal.

En nuestro caso, lo que haremos será buscar un espacio, de componentes principales, de dimensionalidad tal que recoga un porcentaje alto de la variabilidad del espacio de dimensionalidad original. En nuestro caso, este porcentaje lo hemos definido por defecto en el $80$%.

Una vez conseguidos los componentes principales que consiguen recoger el porcentaje marcado de la variabilidad total de los datos originales, establecemos dos estrategias distintas para definir aquellas variables que resultan más relevantes para nuestro análisis por ser más representativas de nuestros datos originales. Estas estrategias son:

- Una primera estrategia, denominada PCA0, que define como variables más relevantes aquellas variables correspondientes a los eigen valores del primer componente que tienen mayor peso sobre este. Este método se basa en el hecho de que el primer componente recoge la mayor variabilidad de los datos originales.

- La segunda estrategia, denominada PCAS, define como variables más relevantes las variables con mayor peso sobre cada uno de los componentes principales que se obtienen al aplicar el análisis de componentes principales.

Estas dos estrategias se han utilizado para realizar distintas selecciones de variables independientes para ejecución de los modelos propuestos.

La siguiente función es utilizada para realizar esta selección de variables relevantes utilizando el análisis **PCA**.

In [1]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

def pca_selection(data, select_vars, percent=0.8):  
    '''
    Selection of most relevant variables based in PCA.
    
    Inputs:
    
        - data: dataframe with timeseries data.
        - select_vars: method used to select most relevant variables from PCs
            'pca0' : select most relevant variables from first PC
            'pcas' : select most relevant variable for every PC
        - percent: percentage of variance explained by PCs 
        
    return list of selected columns as most relevant variables.
    '''    
    
    # In order to get best results all the variables must have a standard distribution
    scaler = preprocessing.StandardScaler()
    data[data.columns] = scaler.fit_transform(data[data.columns])
  
    # Principal component analysis (PCA)
    pca = PCA(percent)
    pca.fit(data)
    X_pca = pca.transform(data)
  
    # get names of most important features
    most_important_names = data.columns
    if select_vars in ['pcas']:  # Select first most important variable of each PC
        # number of components
        n_pcs= pca.components_.shape[0]
        
        # get the index of the most important feature on each component
        most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]
  
        most_important_names = [data.columns[most_important[i]] for i in range(n_pcs)]
  
    else:
  
        # Calculates correlation between each variable with every principal component
        pca_correlation = pca.components_.T * np.sqrt(pca.explained_variance_)
        #loading_matrix = pd.DataFrame(pca_correlation, 
        #                                columns=['PC{}'.format(i) for i in range(n_pcs)], 
        #                                index=data.columns)
        #print(loading_matrix)
  
        if select_vars in ['pca0']: # Select most important features from first component
            pca0_correlation = pca_correlation[:,0]    
            indexes = [i for i in np.where(np.abs(pca0_correlation)>0.5)]
            most_important_names = [data.columns[i] for i in np.unique(np.array(indexes))]
        
        else:  # Select most important features from each component (method not used)         
            indexes = [x for x in np.where(np.abs(pca_correlation)>0.5)]
            most_important_names = [data.columns[i] for i in np.unique(np.array(indexes))]
  
    #mostramos el acumulado de varianza explicada en las nuevas dimensiones
    #plt.plot(np.cumsum(model.explained_variance_ratio_))
    #plt.xlabel('number of components')
    #plt.ylabel('cumulative explained variance')
    #plt.show()
    
    # back data to original distribution
    data[data.columns] = scaler.inverse_transform(data[data.columns])    
    pca = None
    
    return np.unique(most_important_names)

## 3.2. Selección de variables mediante la técnica de Forward Selection (FS)

La siguiente estrategia de selección de variables consiste en ir ejecutando un modelo al que se le van añadiendo variables sucesivamente hasta considerar que se cumple cierta condición. En este caso, partimos de un modelo en el cual únicamente tenemos como variable independiente aquella que tiene mayor correlación con la variable dependiente que pretendemos predecir. Después, en sucesivos bucles se van añadiendo las distintas variables, una a una, al modelo y se evaluan los distintos resultados, quedandonos con aquella nueva variable que genere un mejor resultado. Esta nueva variable se añade a la lista de variables independientes y se vuelve a realizar la misma operación con las variables restantes. Así se continua hasta que se alcanza un criterio establecido de corte. En nuestro caso, este criterio es que la mejora obtenida no supere en un 5% al anterior modelo.

Algunos autores muestran la no idoneidad de esta estrategia para encontrar las variables más representativas de un conjunto de datos, pues extrapolan los resultados de ejecuciones parciales del modelo al conjunto de estos. [Peter Flom (2018)]

La siguiente función realiza la selección de variables representativas para un modelo y variable dependiente dados.

In [ ]:
from math import sqrt
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def fs_selection(data, output, model_func):    
    '''
    Stepwise variable selection by Forward Selection.
    
    Inputs:
    
        - data: dataframe with timeseries data.
        - output: column used as dependant variable (predicted).
        - model_func: model used to predict output values.
        
    return list of selected columns as most relevant variables.
    '''

    # Get model's name
    model_name = model_func.__name__.split('_')[0]
    
    #method : {‘pearson’, ‘kendall’, ‘spearman’}
    corr_db=pd.DataFrame(np.absolute(data.corr(method='pearson')), columns=data.columns, index=data.columns)
    corr_output_db = corr_db[output].sort_values(ascending=False).drop(output)
    
    selected_vars = []
    selected_vars.append(corr_output_db.index[0])
    
    lastBestValue=None
    continueLoop=True
    while continueLoop:
        continueLoop=False
        
        bestValue=None
        selected_var=None
        for index, value in corr_output_db.iteritems():
            if index in selected_vars: continue
            continueLoop=True
            
            x_inputs = []
            x_inputs.extend(selected_vars)
            x_inputs.append(index)

            x_columns = x_inputs
            y_columns = [output]

            # Prepare data for prediction model
            reframed = prepare_data(data_daily, x_columns, y_columns, n_steps_in, n_steps_out)

            # Define scaler as MinMaxScaler between 0 to 1
            scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

            # Scale data
            values = reframed.values.astype('float32')                    

            scaled = scaler.fit_transform(values)

            # Get number of input and output features
            nro_x_columns = len(x_columns)
            nro_y_columns = len(y_columns)
            n_in_features = nro_x_columns*n_steps_in
            n_out_features = nro_y_columns*n_steps_out

            # Get X and y samples
            X = scaled[:,:-n_out_features]
            y = scaled[:,-n_out_features]

            # split into train and test sets
            X_train = X[:int(X.shape[0]*0.8)]
            X_test = X[int(X.shape[0]*0.8):]
            y_train = y[:int(X.shape[0]*0.8)]
            y_test = y[int(X.shape[0]*0.8):]

            samples_train = X_train.shape[0]
            samples_test = X_test.shape[0]

            # Define model
            model = model_func(nro_x_columns, nro_y_columns, n_steps_in, n_steps_out)
 
            reshape_input = model_name not in ["RandomForest","SVM","MLP"]            
            if reshape_input:
                X_train = X_train.reshape((X_train.shape[0], n_steps_in, X_train.shape[1]))
                X_test = X_test.reshape((X_test.shape[0], n_steps_in, X_test.shape[1]))

            # fit model
            if model_name in ["RandomForest", "SVM"]:
                model.fit(X_train, y_train)
                yhat = model.predict(X_test)
                yhat = yhat.reshape((len(yhat), n_out_features))
            else:
                model.fit(X_train, y_train, epochs=10, batch_size=100, 
                          validation_split=0.1, verbose=0, shuffle=False)
                yhat = model.predict(X_test, verbose=0)

            if reshape_input:
                X_test = X_test.reshape((X_test.shape[0], X_test.shape[2]))

            y_test = y_test.reshape((len(y_test), n_out_features))

            # invert scaling for forecast
            inv_yhat = np.concatenate((X_test, yhat), axis=1)
            inv_yhat = scaler.inverse_transform(inv_yhat)
            inv_yhat = inv_yhat[:,-n_out_features]
            # invert scaling for actual
            inv_y = np.concatenate((X_test, y_test), axis=1)
            inv_y = scaler.inverse_transform(inv_y)
            inv_y = inv_y[:,-n_out_features]

            r2 = r2_score(inv_y, inv_yhat) 
            rmse = sqrt(mean_squared_error(inv_y, inv_yhat))

            #print("{} {} {}".format(x_inputs,r2,rmse))
            if not bestValue or r2 > bestValue: # r2 selection criteria
            #if not bestValue or rmse < bestValue: # rmse selection criteria
                bestValue = r2
                selected_var=index
                
            model = None
            #print("%s %s %.2f %.2f" % (output, index, r2, rmse))
            
        if selected_var:
            if lastBestValue and ((lastBestValue * 1.05) >= bestValue): # no 5% improved (r2)
            #if lastBestValue and ((lastBestValue * 0.95) <= bestValue): # no 5% improved (rmse)
                continueLoop=False
            else:
                selected_vars.append(selected_var)
                lastBestValue=bestValue
            
    return selected_vars

# 3. Ajuste de hiperparámetros

Cada uno de los modelos visto anteriormente puede ser parametrizado para encontrar una combinación de parámetros que obtenga los mejores resultados en la regresión. El método que utilizaremos para la búsqueda de una parametrización optima, de cada uno de los modelos, es la búsqueda por malla (*Grid*). Este método consiste en seleccionar una serie de valores a evaluar, para cada uno de los parámetros del modelo. Seguidamente, se evalua el modelo para cada una de las combinaciones posibles de dichos valores quedándonos, al final, con aquella combinación que produce un mejor resultado. En nuestro caso, utilizaremos como criterio de selección el valor de RMSE, aunque también calcularemos el coeficiente de determinación $R^2$ ajustado, que puede también ser usado a tal fin.

El siguiente código tiene como función la de preparar los datos de entrada, dados como series temporales, para la predicción de valores futuros mediante los modelos propuestos anteriormente. Esta función toma como entrada un dataframe con el conjunto total de datos indexado según un campo temporal, una lista de columnas, de nuestro conjunto de datos, utilizadas como variables independientes, una lista de columnas utilizadas como variables dependientes y, un par de parámetros extra que indican el número de pasos hacia atrás dados en la secuencia de variables independientes, *n_steps_in*, y el número de pasos hacia adelante de la secuencia de las variables dependientes, *n_step_out*. 

In [ ]:
def prepare_data(data, x_columns, y_columns, n_steps_in=1, n_steps_out=1, dropnan=True):
    '''
    Prepare time series data to prediction models
    
    Inputs:
    
        - data: dataframe with timeseries data.
        - x_columns: columns to be used as independant variables (predictors).
        - y_columns: columns to be used as dependant variables (predicted).
        - n_steps_in: number of backward steps considered to predictors.
        - n_steps_out: number of forward steps considered to predicted variables.
        - dropnan: whether row containing NaN values must be removed.
        
    return prepared data to prediction models.
    '''
    
    Xdata=data[x_columns]
    Ydata=data[y_columns]
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_steps_in, 0, -1):
        cols.append(Xdata.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in x_columns]     
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_steps_out):
        cols.append(Ydata.shift(-i))
        if i == 0:
            names += [('%s(t)' % (col)) for col in y_columns]
        else:
            names += [('%s(t+%d)' % (col, i)) for col in y_columns]
    
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        
    return agg

El siguiente código nos va a servir para evaluar todas las combinaciones posibles según diversos métodos de selección de variables independientes, considerando distintas variables dependientes a predecir, sobre los datos de cada una de las estaciones de observación de calidad del aire y empleando, cada uno de los modelos de predicción propuestos. Este código encadena una serie de bucles que nos permiten evaluar todas estas combinaciones para, en el último nivel del bucle, evaluar cada modelo para todas las combinaciones de sus parámetros de configuración considerados. Esto nos generará un conjunto de datos que posteriormente nos servirán para obtener una parametrización para cada modelo, sobre cada estación considerando distintos contaminantes a predecir.

Como ya hemos explicado anteriormente, para la parametrización de los distintos modelos evaluaremos, también, distintas técnicas de selección de variables independientes. En concreto, evaluaremos una técnica de selección de variables por *forward-selection* ($FS$), dos métodos de selección de variables basados en un análisis de componentes principales ($PCA$) y, finalmente, como referencia, ejecutamos cada proceso de parametrización utilizando el conjunto completo de variables independientes.

Los parámetros que trataremos de fijar, para cada uno de los modelos propuestos, son:

- Para el modelo de Random Forest, se fijará la $máxima$ $profundidad$ de los árboles de decisión construidos por el modelo.
- Para el modelo de SVM, se fijan el parámetro de regulación $C$ y el coeficiente $gamma$ del kernel, en este caso se usa el kernel por defecto 'rbf'.
- Para los modelos basados en redes neuronales: $MLP$, $LSTM$ y $GRU$; se fijan el $número$ $de$ $neuronas$ de la única capa oculta, el $batch$ $size$ y el número de $epochs$. 

Nota: la ejecución de este bucle puede requerir de gran cantidad de recursos, además de precisarse de un tiempo elevado de ejecución. Por lo tanto, se aconseja ejecutar dicho bucle por partes, por ejemplo: eligiendo un único método de selección de variables independientes en cada ejecución del bucle. Finalmente se pueden agrupar convenientemente los resultados obtenidos.

In [17]:
import os
import random
from math import sqrt
from statistics import mean
from time import time
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

seed = 7
np.random.seed(seed)

# Models to be parametrized
models = {
        'RandomForest':RandomForest_Model,
        'SVM':SVM_Model,
        'MLP': MLP_Model,
        'LSTM':LSTM_Model,
        'GRU':GRU_Model,
    }

# Air data used as imput: original or imputated
data_inputs = {
#        'original':data_air_hourly,
        'imputated':data_air_hourly_imp
    }

# Stations to be considered (models are parametrized for each stations)
stations=data_air_hourly['CODI_EOI'].unique()
#stations=[8019043, 8019044, 8019054, 8019057]
#stations=[8019044]  

# Dependant variables
outputs=['CO','NOX','NO2','O3']

# Methods for variable selection
vars_selection_methods = [
    'total', # use the whole dataset
    'pca0',  # select most relevant variables from first component in PCA
    'pcas',  # select most relevant variables from most important variables 
             # for each component in PCA
    'fs']    # select most relevant variables using a forward-selection method

# Set backward and forward data used to input and output sequences
n_steps_in = 1
n_steps_out = 1

# Number of times every execution is repeated in order to evaluated randomness
n_iter_search = 1

# specify parameters and distributions to RandomForest Model
rf_param_dist = {
            "max_depth": [2, 5, 10, 20, 50],
            }            
# specify parameters and distributions to SVM Model
svm_param_dist = {
                  "C": range( 1, 500, 10),
                  "gamma": [1.e-9, 1.e-8, 1.e-7, 1.e-6, 1.e-5]
              }
# specify parameters and distributions to neural network models
nn_param_dist = {
            "epochs": [10, 50, 100],
            "batch_size": [10, 50, 100],
            "neurons": [20, 50, 100, 500]
            }

# Total loops
total = len(vars_selection_methods) * \
        len(data_inputs) * \
        len(stations) * \
        len(outputs) * \
        len(models)

# Counter for current loop
count = 0

# Dataframe with complete parametrization data
datas_total = pd.DataFrame()

# Creates folder (if so) to record results
rootTuningFolder = os.path.join(rootDataFolder, 'tuning')
os.makedirs(rootTuningFolder, exist_ok=True)

# Search optimal parameters for every variables selection method
for vars_selection_method in vars_selection_methods:
    print(vars_selection_method)
    
    # Dataframe with parametrization data for current variables selection method
    datas_method = pd.DataFrame()

    # Creates folder (if so) to record partial results
    rootSelectMethodFolder = os.path.join(rootTuningFolder, '{}'.format(vars_selection_method))
    os.makedirs(rootSelectMethodFolder, exist_ok=True)

    # Search optimal parameters for every input dataframe
    for data_input in data_inputs:
        print(data_input)
        data_air = data_inputs[data_input]

        # Search optimal parameters for every air observation station
        for station in stations:
            print(station)

            # Get hourly data for given station
            data_hourly = get_station_data(station, data_air, data_meteo_hourly, data_trafic)
            data_hourly = data_hourly.set_index('DATA').interpolate(method='time', limit_area='inside')         
            data_hourly.dropna(inplace=True)
  
            # Resample data daily
            data_daily = data_hourly.resample('D').mean()
                
            # Initially all the variables are used as input
            x_columns = data_daily.columns    
            x_columns=['CO','NO','NO2','NOX','O3','SO2','DV10','HR','P','PPT','RS','T','VV10','traffic']   
            
            # PCA-based variables selection
            if vars_selection_method in ['pca0', 'pcas']:
                x_columns = pca_selection(data, vars_selection_method)
  
            # Search optimal parameters for every dependant variable
            for output in outputs:  
                y_columns = [output]
                print(y_columns)
  
                # Dataframe with parametrization data for current station
                datas = pd.DataFrame()
                  
                # Search optimal parameters for every evaluated regression model
                for model_name in models:
                    model_func = models[model_name]
  
                    # Forward-Selection
                    if vars_selection_method in ['fs']:
                      x_columns = fs_selection(data_daily, output, model_func)
                
                    print(x_columns)
                    print(model_name)
                
                    # Prepare data for prediction model
                    reframed = prepare_data(data_daily, x_columns, y_columns, n_steps_in, n_steps_out)
                    if reframed.shape[0] > 0: 
        
                        # Define scaler as MinMaxScaler between 0 to 1 
                        scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
  
                        # Scale data (all the variables must be considered in the same way)
                        values = reframed.values.astype('float32')
                        scaled = scaler.fit_transform(values)
  
                        # Get number of input and output features
                        nro_x_columns = len(x_columns)
                        nro_y_columns = len(y_columns)
                        n_in_features = nro_x_columns*n_steps_in
                        n_out_features = nro_y_columns*n_steps_out
                                
                        # Define parameters grid for the selected model
                        if model_name in ["RandomForest"]:
                            grid = ParameterGrid(rf_param_dist)
                        elif model_name in ["SVM"]:
                            grid = ParameterGrid(svm_param_dist)
                        else:
                            grid = ParameterGrid(nn_param_dist)
  
                        cnt = 0 
                        count+=1
        
                        # Total loop per each model (parameter grid)
                        total_partial = len(grid) * n_iter_search
                        count_partial = 0
                
                        # Loop for every parameter combination
                        for p in grid:
                            cnt = cnt+1
                        
                            # Every combination of parameter is executed n times
                            # in order to avoid randomness in the results
                            for i in range(n_iter_search): 
  
                                # Get X and y samples
                                X = scaled[:,:-n_out_features]
                                y = scaled[:,-n_out_features]
                        
                                # split into train and test sets
                                X_train = X[:int(X.shape[0]*0.8)]
                                X_test = X[int(X.shape[0]*0.8):]
                                y_train = y[:int(X.shape[0]*0.8)]
                                y_test = y[int(X.shape[0]*0.8):]
  
                                # number of samples in train and test data
                                samples_train = X_train.shape[0]
                                samples_test = X_test.shape[0]
                                
                                # Define model
                                model = model_func(nro_x_columns, nro_y_columns, n_steps_in, n_steps_out, params=p)
  
                                # fit model and perform prediction for test data, w.r.t selected model
                                if model_name in ["RandomForest", "SVM"]:
  
                                    # Fit model
                                    start_time = time()
                                    model.fit(X_train, y_train)
                                    duration = time() - start_time
  
                                    # Predict y for test data
                                    yhat = model.predict(X_test)  
                                    yhat = yhat.reshape((len(yhat), n_out_features))
  
                                elif model_name in ["MLP"]:
  
                                    # Fit model
                                    start_time = time()
                                    model.fit(X_train, y_train, 
                                              epochs=p["epochs"], 
                                              batch_size=p["batch_size"], 
                                              validation_split=0.1, 
                                              verbose=0, shuffle=False)
                                    duration = time() - start_time
  
                                    # Predict y for test data
                                    yhat = model.predict(X_test, verbose=0)                       
  
                                else:
                                    
                                    X_train = X_train.reshape((X_train.shape[0], n_steps_in, X_train.shape[1]))
                                    X_test = X_test.reshape((X_test.shape[0], n_steps_in, X_test.shape[1]))
  
                                    # Fit model
                                    start_time = time()
                                    model.fit(X_train, y_train, 
                                              epochs=p["epochs"], 
                                              batch_size=p["batch_size"], 
                                              validation_split=0.1, 
                                              verbose=0, shuffle=False)
                                    duration = time() - start_time
  
                                    # Predict y for test data
                                    yhat = model.predict(X_test, verbose=0)
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[2]))
          
                                y_test = y_test.reshape((len(y_test), n_out_features))
                              
                                # invert scaling for forecast
                                inv_yhat = np.concatenate((X_test, yhat), axis=1)
                                inv_yhat = scaler.inverse_transform(inv_yhat)
                                inv_yhat = inv_yhat[:,-n_out_features]
                                # invert scaling for actual
                                inv_y = np.concatenate((X_test, y_test), axis=1)
                                inv_y = scaler.inverse_transform(inv_y)
                                inv_y = inv_y[:,-n_out_features]
  
                                # Calculate RMSE and r-squared
                                r2 = r2_score(y_test, yhat) 
                                rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
      
                                # Show progress
                                count_partial+=1 
                                progress = count/total * 100 
                                progress_partial = count_partial/total_partial * 100 
            
                                message = ('Progress: \x1b[32m{:.2f}%\x1b[0m Partial: \x1b[32m{:.2f}%\x1b[0m ' +
                                       'Station: \x1b[31m{}\x1b[0m Selection: \x1b[31m{}\x1b[0m ' +
                                       'Output: \x1b[31m{}\x1b[0m Model: \x1b[31m{}\x1b[0m ' +
                                       'Params: \x1b[31m{}\x1b[0m Iter: \x1b[31m{}\x1b[0m ' +
                                       'RMSE: \x1b[31m{:.2f}\x1b[0m Duration: \x1b[32m{:.2f}\x1b[0m \r')\
                                       .format(progress, progress_partial, station, vars_selection_method
                                               , y_columns, model_name, p, i, rmse, duration)
                                
                                if not rootDataFolder.startswith('/content/drive'):
                                    sys.stdout.write(300*' '+'\r')
                                    sys.stdout.write(message)
                                    sys.stdout.flush()
                                else:
                                    print(message)
                
                                # Record partial results
                                row = {}
                                row['CODI_EOI'] = [station]            
                                row['Data'] = [data_input]         
                                row['Samples_train'] = [samples_train] 
                                row['Samples_test'] = [samples_test]
                                row['Model'] = [model_name]
                                row['params'] = [p]
                                row['Duration'] = [duration]
                                row['count'] = [cnt]
                                row['iter'] = [i]
                                row['R2'] = [r2]
                                row['RMSE'] = [rmse]
                                row['vars_selection_method'] = [vars_selection_method]   
                                row['x_columns'] = [x_columns]
                                row['y_columns'] = [y_columns]
                                datas = datas.append(pd.DataFrame.from_dict(row), ignore_index=True, sort=False)
                                
                                X_train, y_train, X_test, y_test = None, None, None, None
                                X, y = None, None
                                model = None
  
                datas.to_csv(os.path.join(rootSelectMethodFolder, 'tuning_{}_{}.csv'.format(station, output)), index = False)
                datas_method = datas_method.append(datas, ignore_index=True)

    datas_method.to_csv(os.path.join(rootSelectMethodFolder, 'tuning.csv'), index = False)
    datas_total = datas_total.append(datas_method, ignore_index=True)

datas_total.to_csv(os.path.join(rootTuningFolder, 'tuning.csv'), index = False)

fs
8019044
Progress: 10.00% Partial: 20.00% Station: 8019044 Selection: fs Output: ['NO2'] Model: RandomForest Params: {'max_depth': 2} Iter: 0 RMSE: 15.84 Duration: 0.00 
Progress: 10.00% Partial: 40.00% Station: 8019044 Selection: fs Output: ['NO2'] Model: RandomForest Params: {'max_depth': 5} Iter: 0 RMSE: 14.82 Duration: 0.00 
Progress: 10.00% Partial: 60.00% Station: 8019044 Selection: fs Output: ['NO2'] Model: RandomForest Params: {'max_depth': 10} Iter: 0 RMSE: 19.33 Duration: 0.00 
Progress: 10.00% Partial: 80.00% Station: 8019044 Selection: fs Output: ['NO2'] Model: RandomForest Params: {'max_depth': 20} Iter: 0 RMSE: 20.48 Duration: 0.00 
Progress: 10.00% Partial: 100.00% Station: 8019044 Selection: fs Output: ['NO2'] Model: RandomForest Params: {'max_depth': 50} Iter: 0 RMSE: 21.39 Duration: 0.00 
Progress: 20.00% Partial: 0.40% Station: 8019044 Selection: fs Output: ['NO2'] Model: SVM Params: {'C': 1, 'gamma': 1e-09} Iter: 0 RMSE: 19.08 Duration: 0.02 
Progress: 20.00% Part

Incluimos, a continuación, un trozo de código que nos permitirá, al final del todo, realizar un agrupamiento de todas las posibles ejecuciones del código anterior.

In [ ]:
# Dataframe with complete parametrization data
datas = pd.DataFrame()

# Loop for each selection of independant variables
for vars_selection_method in ['total', 'pca0', 'pcas', 'fs']:
    
    # Read results for selection of independant variables
    datas_method = pd.read_csv(os.path.join(rootDataFolder, 'tuning/{}/tuning.csv'.format(vars_selection_method)), sep=',', encoding = "UTF-8")
    
    # Append data to global results
    datas_total = datas_total.append(datas_method, ignore_index=True)

# Save grouped results
datas.to_csv(os.path.join(rootDataFolder, 'tuning/tuning.csv'), index = False)

Forzamos lectura del dataframe con los resultados globales obtenidos anteriormente.

In [ ]:
filename = os.path.join(rootDataFolder, 'tuning/tuning.csv')
datas = pd.read_csv(filename, sep=',', encoding = "UTF-8")

A continuación, vemos la mejor opción para cada una de las estaciones, modelo de predicción y variable predicha o dependiente (en otro notebook posterior se incluye un análisis más completo de estos resultados):

In [22]:
datas.sort_values(['CODI_EOI']) \
    .groupby(['CODI_EOI','Model','y_columns'], sort=False) \
    .apply(lambda x: x.sort_values(['RMSE']).head(1)) \
    .reset_index(drop=True)    

,CODI_EOI,Data,Samples_train,Samples_test,Model,params,Duration,count,iter,R2,RMSE,vars_selection_method,x_columns,y_columns
0,8019043,imputated,753,189,GRU,"{'batch_size': 10, 'epochs': 100, 'neurons': 500}",96.650924,12,0,0.735196,11.008632,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10'...",['O3']
1,8019043,imputated,753,189,SVM,"{'C': 491, 'gamma': 1e-05}",0.017572,250,0,0.177558,16.061630,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10'...",['NO2']
2,8019043,imputated,753,189,MLP,"{'batch_size': 50, 'epochs': 100, 'neurons': 20}",2.219121,21,0,0.567364,11.649254,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10'...",['NO2']
3,8019043,imputated,753,189,GRU,"{'batch_size': 50, 'epochs': 50, 'neurons': 50}",3.590017,18,0,0.507354,12.430948,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10'...",['NO2']
4,8019043,imputated,753,189,LSTM,"{'batch_size': 50, 'epochs': 100, 'neurons': 20}",4.348256,21,0,0.516570,12.314131,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10'...",['NO2']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,8019057,imputated,753,189,RandomForest,{'max_depth': 2},0.003683,1,0,0.272656,0.087156,pca0,['CO' 'NO' 'NO2' 'NOX' 'O3'],['CO']
76,8019057,imputated,753,189,RandomForest,{'max_depth': 2},0.002450,1,0,0.337758,23.032872,pca0,['CO' 'NO' 'NO2' 'NOX' 'O3'],['NOX']
77,8019057,imputated,753,189,SVM,"{'C': 491, 'gamma': 1e-05}",0.007794,250,0,0.323478,23.279865,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10'...",['NOX']
78,8019057,imputated,753,189,MLP,"{'batch_size': 100, 'epochs': 10, 'neurons': 500}",0.561581,28,0,0.468557,20.633256,total,"['CO', 'NO', 'NO2', 'NOX', 'O3', 'SO2', 'DV10'...",['NOX']


# Bibliografía

- Gironés J, Casas J, Minguillón J, Caihuelas R. Minería de datos - modelos y algoritmos. Primera edición ed.: Editorial UOC; 2017.

- Gladilin Peter and Maria Matskevichus (2019). Hyperparameters Tuning for Machine Learning Models for Time Series Forecasting. 2019 Sixth International Conference on Social Networks Analysis, Management and Security (SNAMS). 

- Simone Centellegher (2020) . [How to compute PCA loadings and the loading matrix with scikit-learn](https://scentellegher.github.io/machine-learning/2020/01/27/pca-loadings-sklearn.html)

- https://stats.stackexchange.com/questions/108148/pull-out-most-important-variables-from-pca

- Wikipedia. Stepwise regression. https://en.wikipedia.org/wiki/Stepwise_regression

- Wikipedia. Multilayer perceptron. https://en.wikipedia.org/wiki/Multilayer_perceptron

- Wikipedia. Long short-term memory. https://en.wikipedia.org/wiki/Long_short-term_memory

- Wikipedia. Gated recurrent unit. https://en.wikipedia.org/wiki/Gated_recurrent_unit

- Peter Flom (2018). Stopping stepwise: Why stepwise selection is bad and what you should use instead. https://towardsdatascience.com/stopping-stepwise-why-stepwise-selection-is-bad-and-what-you-should-use-instead-90818b3f52df

- Frank Harrell (2001). Regression Modeling Strategies. Chapter 5. Resampling, Validating, Describing, and Simplifying the Model. 

